# Part A: Braitenberg Behaviors with Keyboard Control

This notebook implements multiple Braitenberg behaviors:
- **Love**: Follow humans, maintain optimal distance
- **Fear**: Retreat from humans
- **Aggressive**: Approach humans quickly
- **Curious**: Explore and investigate humans

Use keyboard to switch behaviors:
- Press 'l' for Love
- Press 'f' for Fear
- Press 'a' for Aggressive
- Press 'c' for Curious
- Press 's' to Stop

**Step 1: Create display widgets**

In [ ]:
import ipywidgets.widgets as widgets
from IPython.display import display
import cv2

# Create widgets for displaying images
display_color = widgets.Image(format='jpeg', width='45%')
display_depth = widgets.Image(format='jpeg', width='45%')
layout = widgets.Layout(width='100%')

sidebyside = widgets.HBox([display_color, display_depth], layout=layout)
display(sidebyside)

# Convert numpy array to jpeg coded data for displaying
def bgr8_to_jpeg(value):
    return bytes(cv2.imencode('.jpg', value)[1])

**Step 2: Load YOLO model for human detection**

In [ ]:
from ultralytics import YOLO

# Load the YOLO model
model = YOLO("yolo11l_half.engine")

**Step 3: Initialize camera system**

In [ ]:
import traitlets
import numpy as np
import pyzed.sl as sl
import threading
import motors
from traitlets.config.configurable import SingletonConfigurable

# Initialize the Robot class
robot = motors.MotorsYukon(mecanum=False)

class Camera(SingletonConfigurable):
    color_value = traitlets.Any()  # Monitor color_value variable
    
    def __init__(self):
        super(Camera, self).__init__()

        self.zed = sl.Camera()
        init_params = sl.InitParameters()
        init_params.camera_resolution = sl.RESOLUTION.VGA
        init_params.depth_mode = sl.DEPTH_MODE.ULTRA
        init_params.coordinate_units = sl.UNIT.MILLIMETER

        status = self.zed.open(init_params)
        if status != sl.ERROR_CODE.SUCCESS:
            print("Camera Open : "+repr(status)+". Exit program.")
            self.zed.close()
            exit(1)

        self.runtime = sl.RuntimeParameters()
        self.thread_runnning_flag = False

        camera_info = self.zed.get_camera_information()
        self.width = camera_info.camera_configuration.resolution.width
        self.height = camera_info.camera_configuration.resolution.height
        self.image = sl.Mat(self.width, self.height, sl.MAT_TYPE.U8_C4, sl.MEM.CPU)
        self.depth = sl.Mat(self.width, self.height, sl.MAT_TYPE.F32_C1, sl.MEM.CPU)
        self.point_cloud = sl.Mat(self.width, self.height, sl.MAT_TYPE.F32_C4, sl.MEM.CPU)

    def _capture_frames(self):
        while(self.thread_runnning_flag == True):
            if self.zed.grab(self.runtime) == sl.ERROR_CODE.SUCCESS:
                self.zed.retrieve_image(self.image, sl.VIEW.LEFT)
                self.zed.retrieve_measure(self.depth, sl.MEASURE.DEPTH)
                self.color_value_BGRA = self.image.get_data()
                self.color_value = cv2.cvtColor(self.color_value_BGRA, cv2.COLOR_BGRA2BGR)
                self.depth_image = np.asanyarray(self.depth.get_data())
                
    def start(self):
        if self.thread_runnning_flag == False:
            self.thread_runnning_flag = True
            self.thread = threading.Thread(target=self._capture_frames)
            self.thread.start()

    def stop(self):
        if self.thread_runnning_flag == True:
            self.thread_runnning_flag = False
            self.thread.join()
            robot.stop()

camera = Camera()
camera.start()

**Step 4: Implement all behavior functions**

In [ ]:
# Global variable to store current behavior mode
current_behavior = 'stop'  # Options: 'love', 'fear', 'aggressive', 'curious', 'stop'

def detect_human(frame):
    """Common function to detect humans and return closest human info"""
    results = model(frame, verbose=False)
    
    conf_threshold = 0.5
    human_detected = False
    min_distance = float('inf')
    closest_bbox = None
    
    for result in results:
        for i in range(len(result.boxes.cls)):
            if result.boxes.cls[i] == 0:  # Human subject
                if result.boxes.conf[i] > conf_threshold:
                    human_detected = True
                    bbox = result.boxes.xyxy[i]
                    
                    # Calculate center of bounding box
                    center_x = int((bbox[0] + bbox[2]) / 2)
                    center_y = int((bbox[1] + bbox[3]) / 2)
                    
                    # Get depth at human center
                    if (0 <= center_y < camera.height and 0 <= center_x < camera.width):
                        distance = camera.depth_image[center_y, center_x]
                        
                        if not np.isnan(distance) and distance > 0:
                            if distance < min_distance:
                                min_distance = distance
                                closest_bbox = bbox
    
    return human_detected, min_distance, closest_bbox

def love_behavior(frame, depth_colormap, human_detected, distance, bbox):
    """Love behavior - follow human and maintain optimal distance"""
    if human_detected and distance != float('inf'):
        # Draw pink box
        cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), 
                     (int(bbox[2]), int(bbox[3])), (255, 105, 180), 2)
        
        bbox_center_x = (bbox[0] + bbox[2]) / 2
        frame_center_x = camera.width / 2
        
        if distance < 450:
            robot.backward(0.2)
            cv2.putText(depth_colormap, f'LOVE: Too close {distance:.0f}mm', 
                       (150, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
        elif distance >= 450 and distance <= 600:
            robot.stop()
            cv2.putText(depth_colormap, f'LOVE: Perfect {distance:.0f}mm <3', 
                       (150, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 255), 2)
        elif distance > 600 and distance <= 1200:
            if bbox_center_x < frame_center_x - 50:
                robot.left(0.3)
            elif bbox_center_x > frame_center_x + 50:
                robot.right(0.3)
            else:
                robot.forward(0.3)
            cv2.putText(depth_colormap, f'LOVE: Approaching {distance:.0f}mm', 
                       (150, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 180, 0), 2)
        else:
            if bbox_center_x < frame_center_x - 50:
                robot.left(0.4)
            elif bbox_center_x > frame_center_x + 50:
                robot.right(0.4)
            else:
                robot.forward(0.5)
            cv2.putText(depth_colormap, f'LOVE: Coming {distance:.0f}mm', 
                       (150, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 100, 0), 2)
    else:
        robot.stop()
        cv2.putText(depth_colormap, 'LOVE: Waiting...', 
                   (200, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (128, 128, 128), 2)

def fear_behavior(frame, depth_colormap, human_detected, distance, bbox):
    """Fear behavior - retreat from humans"""
    if human_detected and distance != float('inf'):
        # Draw red box
        cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), 
                     (int(bbox[2]), int(bbox[3])), (0, 0, 255), 2)
        
        bbox_center_x = (bbox[0] + bbox[2]) / 2
        frame_center_x = camera.width / 2
        human_on_left = bbox_center_x < frame_center_x
        
        if distance < 500:
            robot.backward(0.6)
            cv2.putText(depth_colormap, f'FEAR: DANGER {distance:.0f}mm!', 
                       (150, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
        elif distance < 900:
            robot.backward(0.4)
            cv2.putText(depth_colormap, f'FEAR: Caution {distance:.0f}mm', 
                       (150, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 165, 255), 2)
        elif distance < 1300:
            robot.backward(0.2)
            cv2.putText(depth_colormap, f'FEAR: Alert {distance:.0f}mm', 
                       (150, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
        else:
            robot.stop()
            cv2.putText(depth_colormap, f'FEAR: Safe {distance:.0f}mm', 
                       (150, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    else:
        robot.stop()
        cv2.putText(depth_colormap, 'FEAR: No threat', 
                   (200, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

def aggressive_behavior(frame, depth_colormap, human_detected, distance, bbox):
    """Aggressive behavior - approach humans quickly and directly"""
    if human_detected and distance != float('inf'):
        # Draw orange box
        cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), 
                     (int(bbox[2]), int(bbox[3])), (0, 165, 255), 2)
        
        bbox_center_x = (bbox[0] + bbox[2]) / 2
        frame_center_x = camera.width / 2
        
        if distance < 300:
            robot.stop()
            cv2.putText(depth_colormap, f'AGGRESSIVE: Target reached {distance:.0f}mm', 
                       (100, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
        else:
            # Always approach at high speed
            if bbox_center_x < frame_center_x - 50:
                robot.left(0.5)
            elif bbox_center_x > frame_center_x + 50:
                robot.right(0.5)
            else:
                robot.forward(0.7)
            cv2.putText(depth_colormap, f'AGGRESSIVE: Charging {distance:.0f}mm!', 
                       (120, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 165, 255), 2)
    else:
        robot.stop()
        cv2.putText(depth_colormap, 'AGGRESSIVE: Searching...', 
                   (180, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (128, 128, 128), 2)

def curious_behavior(frame, depth_colormap, human_detected, distance, bbox):
    """Curious behavior - approach cautiously then circle around"""
    if human_detected and distance != float('inf'):
        # Draw cyan box
        cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), 
                     (int(bbox[2]), int(bbox[3])), (255, 255, 0), 2)
        
        bbox_center_x = (bbox[0] + bbox[2]) / 2
        frame_center_x = camera.width / 2
        
        if distance < 400:
            # Circle around when close
            robot.forward(0.2)
            robot.right(0.3)
            cv2.putText(depth_colormap, f'CURIOUS: Investigating {distance:.0f}mm', 
                       (120, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
        elif distance < 800:
            # Approach slowly
            if bbox_center_x < frame_center_x - 50:
                robot.left(0.2)
            elif bbox_center_x > frame_center_x + 50:
                robot.right(0.2)
            else:
                robot.forward(0.2)
            cv2.putText(depth_colormap, f'CURIOUS: Approaching {distance:.0f}mm', 
                       (120, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 100), 2)
        else:
            # Approach at moderate speed
            if bbox_center_x < frame_center_x - 50:
                robot.left(0.3)
            elif bbox_center_x > frame_center_x + 50:
                robot.right(0.3)
            else:
                robot.forward(0.4)
            cv2.putText(depth_colormap, f'CURIOUS: Interested {distance:.0f}mm', 
                       (140, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 200, 0), 2)
    else:
        # Explore when no human detected
        robot.forward(0.2)
        robot.right(0.2)
        cv2.putText(depth_colormap, 'CURIOUS: Exploring...', 
                   (180, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (128, 128, 128), 2)

**Step 5: Main behavior callback with keyboard control**

In [ ]:
# Create text input widget for keyboard control
text_input = widgets.Text(
    value='',
    placeholder='Type: l=Love, f=Fear, a=Aggressive, c=Curious, s=Stop',
    description='Behavior:',
    disabled=False
)

def on_text_change(change):
    """Handle keyboard input to switch behaviors"""
    global current_behavior
    input_value = change['new']
    
    if len(input_value) > 0:
        last_char = input_value[-1].lower()
        
        if last_char == 'l':
            current_behavior = 'love'
            print('\nSwitched to LOVE behavior')
        elif last_char == 'f':
            current_behavior = 'fear'
            print('\nSwitched to FEAR behavior')
        elif last_char == 'a':
            current_behavior = 'aggressive'
            print('\nSwitched to AGGRESSIVE behavior')
        elif last_char == 'c':
            current_behavior = 'curious'
            print('\nSwitched to CURIOUS behavior')
        elif last_char == 's':
            current_behavior = 'stop'
            robot.stop()
            print('\nSTOPPED - Robot halted')

text_input.observe(on_text_change, names='value')
display(text_input)

def main_callback(change):
    """Main callback that executes current behavior"""
    frame = change['new']
    
    # Prepare depth colormap for display
    depth_colormap = cv2.applyColorMap(
        cv2.convertScaleAbs(camera.depth_image, alpha=0.03), 
        cv2.COLORMAP_JET)
    
    # Detect human
    human_detected, distance, bbox = detect_human(frame)
    
    # Execute behavior based on current mode
    if current_behavior == 'love':
        love_behavior(frame, depth_colormap, human_detected, distance, bbox)
    elif current_behavior == 'fear':
        fear_behavior(frame, depth_colormap, human_detected, distance, bbox)
    elif current_behavior == 'aggressive':
        aggressive_behavior(frame, depth_colormap, human_detected, distance, bbox)
    elif current_behavior == 'curious':
        curious_behavior(frame, depth_colormap, human_detected, distance, bbox)
    else:  # stop
        robot.stop()
        cv2.putText(depth_colormap, 'STOPPED', 
                   (250, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (128, 128, 128), 2)
    
    # Display current behavior mode on color image
    cv2.putText(frame, f'Mode: {current_behavior.upper()}', 
               (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    
    # Display images
    scale = 0.1
    resized_color = cv2.resize(frame, None, fx=scale, fy=scale, 
                               interpolation=cv2.INTER_AREA)
    resized_depth = cv2.resize(depth_colormap, None, fx=scale, fy=scale, 
                               interpolation=cv2.INTER_AREA)
    display_color.value = bgr8_to_jpeg(resized_color)
    display_depth.value = bgr8_to_jpeg(resized_depth)

# Attach callback to camera
camera.observe(main_callback, names=['color_value'])

print('\n=== Braitenberg Behaviors Active ===')
print('Press keys to switch behaviors:')
print('  l = Love (follow human)')
print('  f = Fear (retreat from human)')
print('  a = Aggressive (charge at human)')
print('  c = Curious (investigate human)')
print('  s = Stop (halt robot)')
print('===================================')

**To stop the system:**

In [ ]:
camera.stop()
print('System stopped - camera and robot halted')

## Behavior Descriptions:

### Love Behavior
- Follows detected humans
- Maintains optimal distance (450-600mm)
- Retreats if too close, approaches if too far
- Turns to keep human centered

### Fear Behavior
- Retreats from detected humans
- Faster retreat when human is closer
- Graduated response: danger < 500mm, caution < 900mm, alert < 1300mm
- Stops when human is far enough

### Aggressive Behavior
- Charges at detected humans at high speed
- Direct approach with minimal hesitation
- Stops only when very close (< 300mm)
- Quickly reorients to target

### Curious Behavior
- Approaches humans cautiously
- Circles around when close to investigate
- Explores environment when no human present
- Moderate speeds with investigative movements